In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
import random
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import gzip
import json

In [2]:
raw_data = pd.read_csv("Data/Gift_Cards.csv", names=['Item', 'User', 'Rating', 'Timestamp'])
data = raw_data.sample(n=30000)
data.drop(['Timestamp'], axis=1,inplace=True)
data = data[['User', 'Item', 'Rating']]

In [3]:
data

,User,Item,Rating
104077,AJ9HNBJC9L7Q9,B00JDQJZWG,5.0
50472,AUGMEPTP2YCWR,B0080IR6BA,5.0
127516,A2DIYF3IYOHLE1,B01B25M52W,5.0
57735,AYBAQQJCJWFU5,B0091JKY0M,5.0
34522,A23SMORBTNWZAY,B005EISOUQ,5.0
...,...,...,...
36599,A2M3WEW0IXLLXE,B005ESMJ02,5.0
116506,A18ZNN8YIGSH63,B00PMLDTO6,5.0
81503,A2WBAB0253CCBY,B00CHSWGFU,5.0
112130,A4J2JZV7UY37V,B00O3YIGOG,5.0


In [4]:
new_df=data.head(10000)
ratings_matrix = new_df.pivot_table(values='Rating', index='User', columns='Item', fill_value=0)
ratings_matrix.head()

Item,B001GXRQW0,B001H53QE4,B001H53QEO,B002BSHDJK,B002DN7XS4,B002O018DM,B002OOBESC,B002PY04EG,B002QFXC7U,B002VG4BRO,...,B01GF6X842,B01GF7GNCA,B01GKWEH64,B01GKWEJTO,B01GKWEPBG,B01GKZ37SA,B01GKZ3LH2,B01GKZ3SQG,B01GP1W4LA,B01H5PPJT4
User,,,,,,,,,,,,,,,,,,,,,
A0173750PFTTV2DMHIPE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0394918L2YIZXYSU18E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0490453AYGB1BAAOG64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A06018559UAYTKQCA99D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0661118TXRAOKLYQN6R,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = ratings_matrix.T
X1 = X
X.head()

User,A0173750PFTTV2DMHIPE,A0394918L2YIZXYSU18E,A0490453AYGB1BAAOG64,A06018559UAYTKQCA99D,A0661118TXRAOKLYQN6R,A0807760H1KAYIBFPIS,A09225229MO7JS8WEWGW,A094225671L9CQ4YBDTB,A0962950K4K3R28UIBBB,A100DDXFFPBIYK,...,AZXOHF1I7VFFD,AZY34UY11PNZJ,AZY5EY0B860SN,AZYA8C4MU2ZSC,AZYCZUXOS9POB,AZYFQXCHLQH83,AZYTF3HP63M8O,AZZ2BGTZISUK0,AZZ46ACWAAPHV,AZZM5MUOG0LRK
Item,,,,,,,,,,,,,,,,,,,,,
B001GXRQW0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B001H53QE4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B001H53QEO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002BSHDJK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002DN7XS4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(925, 10)

In [7]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(925, 925)

In [8]:
i = X.index[random.randint(0, np.size(correlation_matrix, 0))]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

174

In [9]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(925,)

In [10]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:24]

['B002BSHDJK',
 'B0046IIHMK',
 'B004KNWWOK',
 'B004KNWWPE',
 'B004KNWWPY',
 'B004KNWWRC',
 'B004KNWWU4',
 'B004KNWWUE',
 'B004KNWWUY',
 'B004KNWWZE',
 'B004KNWX08',
 'B004KNWX0S',
 'B004KNWX2Q',
 'B004KNWX6C',
 'B004KNWX7G',
 'B004KNWX80',
 'B004KNWX8U',
 'B004KNWXA8',
 'B004LLIKUQ',
 'B004LLIL3C',
 'B004LLIL3M',
 'B004LLIL5K',
 'B004LLIL5U',
 'B004LLIL6Y']

In [11]:
def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_Gift_Cards.json.gz')
data_set = review_data[:30000]
data_set
    

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Gift Cards, Gift Cards]",,[Gift card for the purchase of goods or servic...,,Serendipity 3 $100.00 Gift Card,[],[],,Serendipity 3,[],[],[],{' Product Dimensions: ': '3.4 x 2.1 ...,Grocery,,,,B001BKEWF2
1,"[Gift Cards, Gift Cards]",,[Amazon.com Gift Cards are the perfect way to ...,,Amazon.com Gift Cards,[],[https://images-na.ssl-images-amazon.com/image...,,Amazon,[Amazon.com Gift cards never expire and carry ...,[],"[BT00DC6QU4, B01I4AHZXC, B0719C5P56, B01K8RLHZ...","{'Shipping Weight:': '0.5 ounces', 'Domestic S...",Gift Cards,,,,B001GXRQW0
2,"[Gift Cards, Gift Cards]",,"[<div class=""aplus""> <br>Amazon.com Gift Cards...",,"Amazon.com Gift Cards, Pack of 50 (Old Version...","[B005ESMEBQ, B007EE5WNA, B007EE5OBU, B07HJHK8Y...",[https://images-na.ssl-images-amazon.com/image...,,Amazon,"[Contains 50 Gift Cards, Gift Card has no fees...","1,786 in Gift Cards (","[B005ESMEBQ, B007EE5OBU, B007EE5WNA, B007EE60N...","{'Shipping Weight:': '1.8 pounds (', 'Domestic...",Gift Cards,,,"$1,250.00",B001H53QE4
3,"[Gift Cards, Gift Cards]",,"[<div class=""aplus""> <h4>Amazon.com Gift Cards...",,"Amazon.com $50 Gift Cards, Pack of 50 (Old Ve...",[],[https://images-na.ssl-images-amazon.com/image...,,Amazon,"[Contains 50 Gift Cards, Gift Card has no fees...",[],[],"{'Shipping Weight:': '1.6 pounds', 'Domestic S...",Gift Cards,,,,B001H53QEO
4,"[Gift Cards, Gift Cards]",,[],,Wood Puzzle Magic Box Gift Card,[],[https://images-na.ssl-images-amazon.com/image...,,Creative Crafthouse,[],"[>#2,893,775 in Toys & Games (See Top 100 in T...",[],{},Toys & Games,,,,B001KMWN2K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,"[Gift Cards, Gift Cards]",,"[Hand-Pattied Burgers Chicken, Steaks & Fish ...",,Harvey&#39;s Grill and Bar &amp; Hayden&#39;s...,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"5,808 in Gift Cards (",[],"{'Domestic Shipping: ': 'Currently, item can b...",Gift Cards,,,,B01GMFJ8JU
1543,"[Gift Cards, Gift Cards]",,[<strong>Smoothie King Gift Cards. The Perfect...,,Smoothie King Gift Card,"[B00NVUDIZ0, B00BXLVE6Y, B00O4I5S7O, B00GOLGWV...",[https://images-na.ssl-images-amazon.com/image...,,QuickGifts,"[Over 500 Locations Nationwide!, Smoothies, He...","1,201 in Gift Cards (","[B00NVUDIZ0, B00GOLGWVK, B07JKD8G5X, B00BXLVE6...",{'ASIN:': 'B01GOQIKRI'},Gift Cards,,,$10.00,B01GOQIKRI
1544,"[Gift Cards, Gift Cards]",,"[Cattle Baron Restaurants, Inc.is a privately ...",,Cattle Baron Restaurant Gift Card,"[B00CXZPG0O, B00PG84W32]",[https://images-na.ssl-images-amazon.com/image...,,QuickGifts,"[9 Locations over New Mexico and Texas!, Join ...","3,302 in Gift Cards (",[],{'ASIN:': 'B01GOQHGL4'},Gift Cards,,,,B01GOQHGL4
1545,"[Gift Cards, Gift Cards]",,"[The cards, available in $25 increments up to ...",,Go Play Golf Gift Card,"[B077CZY68Z, B07JJHVWXL, B00NVUDIZ0, B01CZ54L6...",[],,Go Play Golf by Fairway Rewards,[Go Play Golf - the only national gift card th...,"1,015 in Gift Cards (","[B01LYIR4CZ, B077CZY68Z, B07736DNZB, B01N9AE2Z...","{'Shipping Weight:': '0.6 ounces (', 'ASIN:': ...",Gift Cards,,,$50.00,B01GP1W4LA


In [12]:
top25 = pd.DataFrame({'ID': Recommend[0:25]})

In [13]:
top25

,ID
0,B002BSHDJK
1,B0046IIHMK
2,B004KNWWOK
3,B004KNWWPE
4,B004KNWWPY
5,B004KNWWRC
6,B004KNWWU4
7,B004KNWWUE
8,B004KNWWUY
9,B004KNWWZE


In [14]:
top25.merge(data_set,left_on = 'ID', right_on = 'asin')

,ID,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,B002BSHDJK,"[Gift Cards, Gift Card Accessories]",,[Attractively keeps the wedding gift cards tog...,,"Wilton Reception Gift Card Holder, White","[B01GV3MWSM, B00G7VPUGU, B00STPPR38, B01A04ORP...",[https://images-na.ssl-images-amazon.com/image...,,Wilton,[12 inch high by 10 inch long by 7 inch deep],"[>#154,492 in Kitchen & Dining (See Top 100 in...",[],{},Amazon Home,,"May 15, 2009",$29.99,B002BSHDJK
1,B0046IIHMK,"[Gift Cards, Gift Cards]",,[Give your friends and family the gift of the ...,,Bonefish Grill Gift Card Collection,[],[],,Bonefish Grill,[Premium greeting cards options are available....,"4,082 in Gift Cards (",[],{'ASIN:': 'B0046IIHMK'},Gift Cards,,,,B0046IIHMK
2,B004KNWWOK,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Queen for a Day,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWOK
3,B004KNWWPE,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Happy Valentine&#3...,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWPE
4,B004KNWWPY,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Loving Giraffes,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWPY
5,B004KNWWRC,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - New Baby Girl (Stork),[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWRC
6,B004KNWWU4,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Happy Birthday (Bi...,[],[],,,[],[],[BT00DC6QU4],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWU4
7,B004KNWWUE,"[Gift Cards, Gift Cards]",,"[<div class=""aplus""><h4>Amazon.com Gift Cards ...",,Amazon Gift Card - Print - Happy Birthday (Cake),[],[],,,[],[],"[BT00DDC7BK, BT00DC6QU4, B01I4AHZXC, B00X4SHPFS]",{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWUE
8,B004KNWWUY,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Amazon Books,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWUY
9,B004KNWWZE,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Happy Father&#39;s...,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWZE


In [15]:
Top25Recommend = top25.merge(data_set,left_on = 'ID', right_on = 'asin')

In [17]:
Top25Recommend[Top25Recommend.columns[:2]]
Top25Recommend = Top25Recommend[Top25Recommend.columns[:2]]

In [18]:
Top25Recommend

,ID,category
0,B002BSHDJK,"[Gift Cards, Gift Card Accessories]"
1,B0046IIHMK,"[Gift Cards, Gift Cards]"
2,B004KNWWOK,"[Gift Cards, Gift Cards, No expiration, no fee..."
3,B004KNWWPE,"[Gift Cards, Gift Cards, No expiration, no fee..."
4,B004KNWWPY,"[Gift Cards, Gift Cards, No expiration, no fee..."
5,B004KNWWRC,"[Gift Cards, Gift Cards, No expiration, no fee..."
6,B004KNWWU4,"[Gift Cards, Gift Cards, No expiration, no fee..."
7,B004KNWWUE,"[Gift Cards, Gift Cards]"
8,B004KNWWUY,"[Gift Cards, Gift Cards, No expiration, no fee..."
9,B004KNWWZE,"[Gift Cards, Gift Cards, No expiration, no fee..."
